In [33]:
import pandas as pd
import numpy as np

In [34]:
dataset_df = pd.read_csv("/content/dataset.csv")

In [35]:
dataset_df.head(10)

,Unnamed: 0,harmful,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation
5,5,0,[2/2] huge fan fare and big talking before the...
6,6,0,@user camping tomorrow @user @user @user @use...
7,7,0,the next school year is the year for exams.ð...
8,8,0,we won!!! love the land!!! #allin #cavs #champ...
9,9,0,@user @user welcome here ! i'm it's so #gr...


In [36]:
dataset_df.drop(columns=["Unnamed: 0"])

,harmful,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
56740,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
56741,1,"you've gone and broke the wrong heart baby, an..."
56742,1,young buck wanna eat!!.. dat nigguh like I ain...
56743,1,youu got wild bitches tellin you lies


In [37]:
dataset_df.shape

(56745, 3)

In [38]:
dataset_df['harmful'].value_counts()

,count
harmful,
0,32592
1,24153


In [39]:
for _ in range(10):
  random_index = np.random.randint(0, len(dataset_df))
  random_data = dataset_df.iloc[random_index]
  print(random_data["tweet"], random_data["harmful"])

save $$ no logins x brokers   #me #change #memes #love   #education #university  0
 @user @user @user so   all these little pundits running around 5 moves behind @user he is theâ¦ 0
best #essentialoils for #anxiety !!     #healthy   #peace !!  #altwaystoheal !!  0
 @user one day to go until the the wedding awards with the magic mirror. @user  #imperialwarmuseum    0
@kieffer_jason @zach_smith98 @NathannDevlin so why the fuck are still talking shit. You acting like a pussy ass nigga 1
Megatron must've fucked Stafford bitch or something.. He will not throw my mans the ball. 1
@MarkHalperin @WSJopinion @BarackObama Anything to trash this black President!! 1
RT @_BeautifulKeezy: &#8220;@100046729: @_BeautifulKeezy lmfao bitch was I talking to you &#128514;&#128514;&#8221;but I'm talking to you in my bubba voice &#128514;&#128514;&#128514;&#128128; 1
@CassiusClay_232 who all in that hoe 1
Just purchased my first trenta iced coffee....this is a slippery slope 0


Preprocessing

In [40]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [41]:
x = dataset_df["tweet"].values
y = dataset_df["harmful"].values

# The .values attribute converts the pandas Series (each column in a DataFrame) into a NumPy array. This is useful when you need to perform operations that require NumPy arrays or when you need to pass the data into machine learning models or other libraries that expect NumPy arrays as input.

In [42]:
print(x)

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"
 '  bihday your majesty' ...
 'young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again'
 'youu got wild bitches tellin you lies'
 '~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB']


In [43]:
sequences = [sequence for sequence in x]
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

In [44]:
type(sequences)

list

In [45]:
type(x)

numpy.ndarray

In [46]:
type(model_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [47]:
import tensorflow as tf

In [48]:
dataset = tf.data.Dataset.from_tensor_slices((model_inputs['input_ids'], y))

In [49]:
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [50]:
# Training pipeline

train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional

model = Sequential(name="tweet-classifier")
model.add(Embedding(len(tokenizer.get_vocab()), 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [56]:
len(tokenizer.get_vocab())

30522

In [57]:
len(tokenizer)

30522

In [60]:
model.summary()

Model: "tweet-classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [61]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [62]:
with tf.device("/device:GPU:0"):
  history = model.fit(train, epochs=1, validation_data=val)

2482/2482 ━━━━━━━━━━━━━━━━━━━━ 95s 35ms/step - loss: 0.2674 - val_loss: 0.1244


In [71]:
# Tokenize the new text
text = "nazi hitler"
new_input = tokenizer(text, padding=True, return_tensors="tf")

# Get the input_ids which is what your model was trained on
input_ids = new_input['input_ids']

# Make prediction
prediction = model.predict(input_ids)


# Print results
print(f"Raw prediction score: {prediction[0][0]}")
print(f"Binary prediction: {'Harmful' if prediction[0][0] > 0.5 else 'Not Harmful'}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw prediction score: 0.6480911374092102
Binary prediction: Harmful


In [72]:
prediction

array([[0.64809114]], dtype=float32)